<a href="https://colab.research.google.com/github/the-chosen-wan/deepfake-detection/blob/main/multi_veiw_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import os
import glob

In [2]:
pow_list=[1,2,4,8,16,32,64,128,256,512,1024,2048]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

def Meso4():
        x = layers.Input(shape = (256,256, 3))
        
        x1 = layers.Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
        x1 = layers.BatchNormalization()(x1)
        x1 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
        x2 = layers.Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
        x2 = layers.BatchNormalization()(x2)
        x2 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
        x3 = layers.Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
        x3 = layers.BatchNormalization()(x3)
        x3 = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
        x4 = layers.Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
        x4 = layers.BatchNormalization()(x4)
        x4 = layers.MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
        y = layers.Flatten()(x4)
        y = layers.Dropout(0.5)(y)
        y = layers.Dense(16)(y)
        y = layers.LeakyReLU(alpha=0.1)(y)
        y = layers.Dropout(0.5)(y)
        y = layers.Dense(1, activation = 'sigmoid')(y)

        return Model(inputs = x, outputs = y)
meso  = Meso4()
meso.load_weights('/content/drive/My Drive/weights/Meso4_F2F.h5')

OSError: ignored

In [ ]:
meso.summary()

In [4]:
class mlp(layers.Layer):
  def __init__(self,units,drop):
    super(mlp,self).__init__()
    self.units = units
    self.drop = drop

    self.dense=[]
    self.drop_layer=layers.Dropout(self.drop)
    for unit in units:
      self.dense.append(layers.Dense(unit,activation='relu'))
  
  def get_config(self):
    config = super(mlp,self).get_config()
    config.update({"drop":self.drop,"units":self.units})
    return config
  
  def call(self,x):

    for i in range(len(self.units)):
      x = self.dense[i](x)
      x = self.drop_layer(x)
    return x


In [5]:
class transformer(layers.Layer):
  def __init__(self,transformer_layers,num_heads):
    super(transformer,self).__init__()
    self.layers = transformer_layers
    self.heads = num_heads
  
  def get_config(self):
     config = super(transformer, self).get_config()
     config.update({"transformer_layers": self.layers,"num_heads":self.heads})
     return config
  
  def build(self,input_shape):
    self.embed_dim = input_shape[2]
    self.units = [2*self.embed_dim,self.embed_dim]
    self.dense=[]
    self.layer_norm1=[]
    self.layer_norm2=[]
    self.mha=[]
    for i in range(self.layers):
      self.dense.append(mlp(self.units,0.2))
      self.layer_norm1.append(layers.LayerNormalization(epsilon=1e-6))
      self.layer_norm2.append(layers.LayerNormalization(epsilon=1e-6))
      self.mha.append(layers.MultiHeadAttention(num_heads=self.heads,key_dim=self.embed_dim))
    
  def call(self,x):

    for i in range(self.layers):
      x1 = self.layer_norm1[i](x)
      x1 = self.mha[i](x1,x1)
      x2 = layers.Add()([x,x1])
      x3 = self.layer_norm2[i](x2)
      x3 = self.dense[i](x3)
      x = layers.Add()([x3,x2]) 
        
    return x

In [6]:
class Patch(layers.Layer):
    def __init__(self,patch_size):
        super(Patch,self).__init__()
        self.patch_size=patch_size
    
    def get_config(self):
        config = super(Patch, self).get_config()
        config.update({"patch_size": self.patch_size})
        return config

    def call(self,patch):
        batch = tf.shape(patch)[0]
        patches = tf.image.extract_patches(
                  images = patch,
                  sizes = [1,self.patch_size,self.patch_size,1],
                  strides = [1,self.patch_size,self.patch_size,1],
                  rates = [1,1,1,1],
                  padding = 'VALID')
        dims = patches.shape[-1]
        patches = tf.reshape(patches,[batch,-1,dims])
        return patches

In [7]:
class inverse_patch(layers.Layer):
  def __init__(self,target_shape,patch_size):
    super(inverse_patch,self).__init__()
    self.H = target_shape[0]
    self.B = target_shape[1]
    self.C = target_shape[2]
    self.patch_size = patch_size
  
  def call(self,x):
    batch_size = tf.shape(x)[0]
    num_patches = tf.shape(x)[1]
    pad = [[0,0],[0,0]]
    p = self.patch_size
    h = self.H
    patches = tf.reshape(x,[batch_size,num_patches,p,p,self.C])
    patches_proc = tf.reshape(patches,[batch_size,h//p,h//p,p*p,self.C])
    patches_proc = tf.split(patches_proc,p*p,3)
    patches_proc = tf.stack(patches_proc,axis=0)
    patches_proc = tf.reshape(patches_proc,[p*p*batch_size,h//p,h//p,self.C])
    reconstructed = tf.compat.v1.batch_to_space_nd(patches_proc,[p, p],pad)
    return reconstructed

In [8]:
class encoding(layers.Layer):
    def __init__(self,num_patches,projection_dim):
        super(encoding,self).__init__()
        self.num = num_patches
        self.projection_dim = projection_dim
        self.dense = layers.Dense(self.projection_dim)
        self.embed = layers.Embedding(input_dim = num_patches , output_dim = self.projection_dim)
    
    def get_config(self):
        config = super(encoding, self).get_config()
        config.update({"num_patches": self.num,"projection_dim":self.projection_dim})
        return config
    
    def call(self,patch):
        pos = tf.range(start = 0,limit = self.num , delta =1)
        return self.dense(patch) + self.embed(pos)

In [9]:
class simple_attention(layers.Layer):
  def __init__(self,projection_dim,pow_list):

    super(simple_attention,self).__init__()
    self.projection_dim=projection_dim
    self.pow_list = pow_list
  
  def get_config(self):
    config = super(simple_attention,self).get_config()
    config.update({"projection_dim":self.projection_dim,"pow_list":self.pow_list})
    return config
    
  def build(self,input_shape):
    H = input_shape[1]
    B = input_shape[2]
    C = input_shape[-1]
    res1 = tf.cast(tf.math.ceil(np.log2(H)),dtype='int32')
    res = self.pow_list[res1]
    self.patch_size=(res//8)
    self.patch_encoder = Patch(self.patch_size)
    p = C*(self.patch_size)**2
    norm = tf.math.sqrt(tf.cast(p,dtype='float32'))


    self.num_patches = ((res)//self.patch_size)**2
    #self.resize = layers.Resizing(res,res)
    self.conv = layers.Conv2D(C,kernel_size=1)
    #self.key_conv = layers.Conv2D(C,kernel_size=1)
    #self.value_conv = layers.Conv2D(C,kernel_size=1)
    self.embed = encoding(self.num_patches,self.projection_dim)
    #self.reshape = inverse_patch((8,8,32),self.patch_size)
    self.sa = transformer(12,4)
    #self.sa = layers.Attention()
    self.normalizer = layers.Lambda(lambda x: x/norm)
  
  def call(self,x):
    #x = self.resize(x1)
    query_block = self.conv(x)
    #key_block = self.key_conv(x)
    #value_block = self.value_conv(x)

    query_block = self.patch_encoder(query_block)
    query_block = self.embed(query_block)

    #key_block = self.patch_encoder(key_block)
    #key_block = self.embed(key_block)

    #value_block = self.patch_encoder(value_block)
    #value_block = self.embed(value_block)

    out = self.sa(query_block)
    #out = self.sa([query_block,value_block,key_block])
    out = self.normalizer(out)
    #out = self.reshape(out)
    return out

In [10]:
backbone_cnn = meso#tf.keras.applications.Xception(include_top = False,weights='imagenet',input_shape=(256,256,3))
layer_list = [2,5,8,11,12]#[0,4,12,22,32,-10]
r=tf.cast(tf.math.ceil(np.log2(22)),dtype='int32')
pow_list[r]

32

In [ ]:
backbone_cnn.layers[22].output

IndexError: ignored

In [11]:
def multi_veiw_attention(embedding_dim):
  heads = []
  l = len(layer_list)
  for i in range(l):
    heads.append(simple_attention(embedding_dim,pow_list))

  out = heads[0]((backbone_cnn.layers[layer_list[0]].output))
  out = tf.expand_dims(out,axis=-1)

  for i in range(1,l):
    t = heads[i](backbone_cnn.layers[layer_list[i]].output)
    t = tf.expand_dims(t,axis=-1)
    out = layers.Concatenate(axis=-1)([out,t])
  
  out = tf.keras.backend.mean(out,axis=-1)
  out = layers.Flatten()(out)
  out = mlp([1024,512],0.5)(out)
  out = layers.Dense(2,activation='softmax')(out)
  return Model(inputs=backbone_cnn.inputs,outputs=out)

In [12]:
final_model = multi_veiw_attention(128)
final_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 8)  224         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 8)  32         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                            

In [ ]:
#GETS TRAIN FILES PATH
train_files = []
label_files= ['fake','real']
for x in os.walk('/content/drive/My Drive/data_fakesecond/train'):
    for y in glob.glob(os.path.join(x[0], '*.jpg')):
        train_files.append(y)
print(len(train_files))

15958


In [ ]:
#GETS VAL FILES PATH
val_files = []
label_files= ['fake','real']
for x in os.walk('/content/drive/My Drive/celeb/test'):
    for y in glob.glob(os.path.join(x[0], '*.jpg')):
        val_files.append(y)
print(len(val_files))

518


In [ ]:
import cv2
#GETS IMAGE ARRAY OUT GIVEN AN IMAGE PATH
def get_input(path):
    im = cv2.imread(path)
    return(im)
 
#CREATES LABEL VECTORE [0,1] OR [1,0] BY EXPLOITING CLASS TYPE label_files IN FILE PATH 
def get_output( path, label_files,mode):
    if mode =='train':
      img_id = path.split('/')[-2]
    else:
      img_id = path.split('/')[-1].split('_')[0].lower()
    laba = []
    for label in label_files:
      if label == img_id:
        laba.append(1)
      else:
        laba.append(0)
    return laba

In [ ]:
#GENERATOR FUNCTION TO PASS THE IMAGES AND LABELS TO model.fit FOR TRAINING
def image_generator_test(files, label_files,mode, resize=(256,256)):
 
  batch_x = []
  batch_y = [] 
          
  for input_path in files:
      input = get_input(input_path)
      output = get_output(input_path, label_files,mode)
      if resize is not None:
        input = cv2.resize(input, resize)
        #input  = input[np.newaxis,:,:,:]
        batch_x.append(input)
        batch_y.append(output)
   
  batch_x = tf.convert_to_tensor(batch_x)
  batch_x = tf.cast(batch_x,dtype='float32')
  batch_x = tf.keras.applications.xception.preprocess_input(batch_x)
  #batch_x = batch_x / 255.0
  batch_y = tf.convert_to_tensor(batch_y)
  #batch_x = patch_encoder(batch_x)

  return batch_x, batch_y

imx,imy = image_generator_test(val_files,label_files,'val')

In [ ]:
#GENERATOR FUNCTION TO PASS THE IMAGES AND LABELS TO model.fit FOR TRAINING
def image_generator(files, label_files, batch_size,mode, resize=(256,256)):
 
      while True:
          batch_paths  = np.random.choice(a  = files, 
                                          size = batch_size)
          batch_x = []
          batch_y = [] 
          
          for input_path in batch_paths:
              input = get_input(input_path)
              output = get_output(input_path, label_files,mode)
              if resize is not None:
                input = cv2.resize(input, resize)
              #input  = input[np.newaxis,:,:,:]
              batch_x.append(input)
              batch_y.append(output)
   
          batch_x = tf.convert_to_tensor(batch_x)
          batch_x = tf.cast(batch_x,dtype='float32')
          batch_x = tf.keras.applications.xception.preprocess_input(batch_x)
          #batch_x = batch_x / 255.0
          batch_y = tf.convert_to_tensor(batch_y)
          #batch_x = patch_encoder(batch_x)

          yield batch_x, batch_y

In [ ]:
final_model.compile('Adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy',tf.keras.metrics.AUC()])
 
checkpoint_filepath = '/content/checkpoint5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, save_weights_only=True, monitor='val_auc_3', mode='max')
 
history=final_model.fit(image_generator(train_files, label_files,mode='train', batch_size = 16), epochs = 30, steps_per_epoch = 40, validation_data = (imx,imy),validation_batch_size=1, callbacks=[model_checkpoint_callback])

Epoch 1/30
40/40 [==============================] - 413s 9s/step - loss: 0.8657 - accuracy: 0.5125 - auc_3: 0.5068 - val_loss: 0.6638 - val_accuracy: 0.6506 - val_auc_3: 0.6788
Epoch 2/30
40/40 [==============================] - 350s 9s/step - loss: 0.7768 - accuracy: 0.5437 - auc_3: 0.5560 - val_loss: 0.7356 - val_accuracy: 0.4614 - val_auc_3: 0.4299
Epoch 3/30
40/40 [==============================] - 340s 9s/step - loss: 0.8505 - accuracy: 0.5063 - auc_3: 0.5029 - val_loss: 0.7981 - val_accuracy: 0.3378 - val_auc_3: 0.3328
Epoch 4/30
40/40 [==============================] - 339s 9s/step - loss: 0.7838 - accuracy: 0.5016 - auc_3: 0.5003 - val_loss: 0.6937 - val_accuracy: 0.4884 - val_auc_3: 0.4981
Epoch 5/30
40/40 [==============================] - 339s 9s/step - loss: 0.7479 - accuracy: 0.4953 - auc_3: 0.5027 - val_loss: 0.6891 - val_accuracy: 0.5656 - val_auc_3: 0.5722
Epoch 6/30
40/40 [==============================] - 338s 9s/step - loss: 0.7311 - accuracy: 0.4797 - auc_3: 0.4837 

KeyboardInterrupt: ignored

In [ ]:
imx,imy = image_generator(val_files,label_files,1,'val')

In [ ]:
final_model = multi_veiw_attention(128)
final_model.load_weights('/content/drive/My Drive/saved/multi_veiw_attention.h5')

In [ ]:
final_model.compile('Adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy',tf.keras.metrics.AUC()])
final_model.evaluate(imx,imy,batch_size=1)

518/518 [==============================] - 46s 68ms/step - loss: 0.8341 - accuracy: 0.4749 - auc_1: 0.4720


[0.8341057300567627, 0.4749034643173218, 0.47204867005348206]

In [ ]:
temp = multi_veiw_attention(128)
temp.load_weights('/content/checkpoint5')
drive.mount('/content/gdrive')
temp.save('/content/gdrive/My Drive/saved/multi_veiw_attention_mesonet.h5')
drive.mount('/content/drive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/drive
